In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import string
import nltk
from nltk.corpus import stopwords

In [2]:
punc = string.punctuation

In [3]:
punc

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

In [6]:
stop = stopwords.words('english')
stop2 = stopwords.words('english')

In [5]:
stop = stopwords.words('english')
[stop.append(x) for x in ['nothing','column','middle','passed','need','lots','many','avenue','take',
                          'area','top','go','else','consider','high','visiting','the','much','would','guess']];

In [6]:
reviews = pd.read_csv('reviews.csv',encoding='latin')

In [7]:
reviews['title'] = (reviews['title']
                    .str.strip()
                    .str.lower()
                    .map(lambda x: ' '.join([j for j in x.split() if j not in stop ])) ## Quitar stopwords
                    .str.replace('(?:\'|\.|[\¿\?\!\¡0-9,;]*)','') # quitar numeros signos de exclamación etcetera
                    .map(lambda x: ' '.join([j for j in x.split() if j not in set(punc)] )) # remover carácteres especiales
                    .str.normalize('NFKD').str.encode('ascii',errors='ignore').str.decode('utf-8') # Sustituir acentos
                    )

In [8]:
reviews['comment'] = (reviews['comment']
                      .str.strip() # Borrar espacios vacíos al primcipio y al final
                      .str.lower()
                      .map(lambda x: ' '.join([j for j in x.split() if j not in stop])) ## Quitar stopwords
                      .str.replace('(?:\'|\.|[\¿\?\!\¡0-9,;]*)','') # quitar numeros signos de exclamación etcetera
                      .map(lambda x: ' '.join([j for j in x.split() if j not in set(punc)] )) # remover carácteres especiales
                      .str.normalize('NFKD').str.encode('ascii',errors='ignore').str.decode('utf-8') # Sustituir acentos
                      )

In [9]:
reviews = (reviews
           .drop('Unnamed: 0',axis=1)
           .assign(sentiment = lambda df: df['rating'].map( lambda x: 1 if x>30 else 0 ))
           )

In [10]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score

In [26]:
# X_train,X_test,y_train,y_test = train_test_split(reviews['comment'],reviews['sentiment'],random_state=0)
X_train,X_test,y_train,y_test = reviews['comment'],reviews['comment'],reviews['sentiment'],reviews['sentiment']
X_vec = CountVectorizer(ngram_range=(1,2)).fit(X_train)
# X_vec = TfidfVectorizer(ngram_range=(1,2)).fit(X_train)
X_train_vec = X_vec.transform(X_train)
model = LogisticRegression()
model.fit(X_train_vec,y_train)
feature_names = np.array(X_vec.get_feature_names())

In [27]:
coefs = model.coef_[0].argsort()
print('smallest coefs: \n{}\n'.format(feature_names[coefs][:10]))
print('largest coefs: \n{}\n'.format(feature_names[coefs][:-11:-1]))

smallest coefs: 
['traffic' 'big' 'interesting' 'busy' 'stops' 'climb' 'crowded' 'market'
 'buses' 'able']

largest coefs: 
['beautiful' 'great' 'history' 'city' 'sunday' 'walk' 'monuments' 'amazing'
 'mexican' 'wonderful']



In [28]:
import gensim
from gensim import corpora

C:\Users\Scastaneda01\AppData\Local\Continuum\anaconda3\lib\site-packages\gensim-3.4.0-py3.6-win-amd64.egg\gensim\utils.py:1197: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [32]:
doc_clean = [doc.split() for doc in X_train]
dictionary=0
dictionary = corpora.Dictionary(doc_clean)
doc_term_matrix = [dictionary.doc2bow(doc) for doc in doc_clean]

## LDA model

El código debajo tarda bastante tiempo en correr.

In [33]:
Lda = gensim.models.LdaModel
ldamodel = Lda(doc_term_matrix, num_topics=3, id2word=dictionary,passes=50)

Los resultados del Topic Modelling no son lo que esperabamos pues no hay tópicos muy evidentes que podamos obtener a partir de las palabras.

¿Cuál creen que haya sido el problema?  
*Noten que muchas de las palabras que aparecen en el tópico 1 están en español.*

¿Cuál creen que sería una solución ?

In [35]:
print(ldamodel.print_topics(num_topics=3,num_words=10)[0])

(0, '0.023*"de" + 0.021*"angel" + 0.020*"reforma" + 0.020*"la" + 0.019*"mexico" + 0.018*"city" + 0.015*"paseo" + 0.014*"beautiful" + 0.013*"walk" + 0.012*"one"')


In [36]:
print(ldamodel.print_topics(num_topics=3,num_words=10)[1])

(1, '0.016*"walk" + 0.014*"people" + 0.014*"great" + 0.013*"place" + 0.012*"street" + 0.011*"reforma" + 0.011*"sunday" + 0.010*"see" + 0.010*"walking" + 0.010*"nice"')


In [37]:
print(ldamodel.print_topics(num_topics=3,num_words=10)[2])

(2, '0.036*"city" + 0.027*"mexico" + 0.015*"reforma" + 0.012*"beautiful" + 0.011*"walk" + 0.009*"nice" + 0.008*"see" + 0.008*"street" + 0.007*"paseo" + 0.007*"chapultepec"')
